## Managing the Chat Conversation History Using Langchain
- To managing the Chat Conversation History Using Langchain we need to makesure that it should not `overflow the Context Window of LLM`.
- We need to add a step that limits the size of messages interacting user will pass.
- `trim_messages` : this helps to reduce how many messages interacting user can send to model. The trimmer allows to specify how many tokens we want to keep, along with other parameters like if we want to always keep the sytem messages and/or whether to allow partial messages

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=GROQ_API_KEY)

In [29]:
from langchain_core .messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy="last", #last : focus on the last conversation
    token_counter=model,
    include_system=True, # include the system messages
    allow_partial=False, # do not allow the partial information
    start_on="human"
)

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a helpful assistance. Answer all the questions to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [31]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# when max_tokens=45, see few lines of HumanMessage and AIMessage will get trim
# Note : SystemMessage will never get trimmed or skipped

messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes")
]

trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes', additional_kwargs={}, response_metadata={})]

In [32]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="what's math question I have asked?")],
        "language":"English"
    }
)

response.content

'You asked a simple math question: 2 + 2.'

In [33]:
# Now wrap up in the Message History :
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store ={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    '''
    session_id will get inherited (use) to BaseChatMessageHistory to create a difference between sessions.
    '''
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [34]:
with_message_history=RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

configurations={"configurable":{"session_id":"chat"}}

In [35]:
response = with_message_history.invoke(
    {
        "messages" : messages + [HumanMessage(content="What's my name?")],
        "language" : "English",
    },
    config=configurations
)

response.content

"You didn't mention your name. I'm happy to chat with you, though!"